In [1]:
import dgl
import numpy as np
import torch
import pandas as pd
import json
from torch.utils.data import DataLoader
import torch.nn as nn
import dgl.nn.pytorch as dglnn
import torch.nn.functional as F

Using backend: pytorch


In [2]:
# import scipy.io
# import urllib.request

# data_url = 'https://data.dgl.ai/dataset/ACM.mat'
# data_file_path = '/tmp/ACM.mat'

# urllib.request.urlretrieve(data_url, data_file_path)
# data = scipy.io.loadmat(data_file_path)
# print(type(data))

In [91]:
houseB = pd.read_csv('/home/harsh/Downloads/Thesis/Graph/KDD20-Hands-on-Tutorial/3-basics/data/houseB.csv')

sensorTime = pd.read_csv('/home/harsh/Downloads/Thesis/Graph/KDD20-Hands-on-Tutorial/3-basics/data/houseB-sensorChangeTime.csv')

### Creating Sensor Timings CSV

In [92]:

length_of_sensor_columns = len(nodesColumns_0)

In [93]:
prev_value_of_sensor_row_list = np.zeros(length_of_sensor_columns, dtype=int)
for i in range(1,len(houseB)):        
    for j in range(length_of_sensor_columns):        
        prev_sensor_value = houseB.iloc[prev_value_of_sensor_row_list[j], 4+j]
        current_sensor_value = houseB.iloc[i, 4+j]

        prev_sensor_time_of_day = houseB.loc[prev_value_of_sensor_row_list[j], 'time_of_the_day']        
        curr_sensor_time_of_day = houseB.loc[i, 'time_of_the_day']
        diff = (curr_sensor_time_of_day - prev_sensor_time_of_day) * 24    
        sensorTime.iloc[i, 4 + j] =  diff
        
        if prev_sensor_value != current_sensor_value:       
            prev_value_of_sensor_row_list[j] = i

In [95]:
sensorTime.to_csv('/home/harsh/Downloads/Thesis/Graph/Our_Graphs/houseB-sensorChangeTime.csv', index = False)

In [184]:
activity_list = []

for i, val in enumerate(np.unique(houseB['activity'].values)):
    d = {}
    d['name'] = val
    d['id'] = i
    activity_list.append(d)
    
print(activity_list)

[{'name': 'brushTeeth', 'id': 0}, {'name': 'eatBreakfast', 'id': 1}, {'name': 'eatDinner', 'id': 2}, {'name': 'getDressed', 'id': 3}, {'name': 'getDrink', 'id': 4}, {'name': 'goToBed', 'id': 5}, {'name': 'idle', 'id': 6}, {'name': 'leaveHouse', 'id': 7}, {'name': 'prepareBreakfast', 'id': 8}, {'name': 'prepareDinner', 'id': 9}, {'name': 'takeShower', 'id': 10}, {'name': 'useToilet', 'id': 11}, {'name': 'washDishes', 'id': 12}]


In [185]:
houseB = houseB[houseB['activity']!='goToBed']
houseB = houseB[houseB['activity']!='leaveHouse']
houseB = houseB[houseB['activity']!='idle']
houseB.groupby('activity').count()


,start,end,time_of_the_day,ToiletDoor_1,Fridge_3,GroceriesCupboard_5,ToiletFlush_6,Frontdoor_7,PlatesCupboard_9,BedroomDoor_10,...,BedroomDresser_18,BathroomPIR_19,PianoSeatPressure_20,SinkFloat_21,ServerNookSeatPressure_22,BalconyDoor_24,KitchenWindow_25,Toaster_26,Microwave_27,KitchenPIR_28
activity,,,,,,,,,,,,,,,,,,,,,
brushTeeth,35,35,35,35,35,35,35,35,35,35,...,35,35,35,35,35,35,35,35,35,35
eatBreakfast,136,136,136,136,136,136,136,136,136,136,...,136,136,136,136,136,136,136,136,136,136
eatDinner,49,49,49,49,49,49,49,49,49,49,...,49,49,49,49,49,49,49,49,49,49
getDressed,37,37,37,37,37,37,37,37,37,37,...,37,37,37,37,37,37,37,37,37,37
getDrink,14,14,14,14,14,14,14,14,14,14,...,14,14,14,14,14,14,14,14,14,14
prepareBreakfast,85,85,85,85,85,85,85,85,85,85,...,85,85,85,85,85,85,85,85,85,85
prepareDinner,89,89,89,89,89,89,89,89,89,89,...,89,89,89,89,89,89,89,89,89,89
takeShower,110,110,110,110,110,110,110,110,110,110,...,110,110,110,110,110,110,110,110,110,110
useToilet,64,64,64,64,64,64,64,64,64,64,...,64,64,64,64,64,64,64,64,64,64


In [186]:
nodes = pd.read_csv('/home/harsh/Downloads/Thesis/Graph/Our_Graphs/nodes.csv')

edges = pd.read_csv('/home/harsh/Downloads/Thesis/Graph/Our_Graphs/bidrectional_edges.csv')

In [187]:
u = edges['Src']
v = edges['Dst']

In [188]:
 config = {
     "ActivityIdList" :
   [{'name': 'brushTeeth', 'id': 0}, {'name': 'eatBreakfast', 'id': 1}, {'name': 'eatDinner', 'id': 2}, {'name': 'getDressed', 'id': 3}, {'name': 'getDrink', 'id': 4}, {'name': 'goToBed', 'id': 5}, {'name': 'idle', 'id': 6}, {'name': 'leaveHouse', 'id': 7}, {'name': 'prepareBreakfast', 'id': 8}, {'name': 'prepareDinner', 'id': 9}, {'name': 'takeShower', 'id': 10}, {'name': 'useToilet', 'id': 11}, {'name': 'washDishes', 'id': 12}]
}

In [189]:
def getIDFromClassName(train_label):
    ActivityIdList = config['ActivityIdList']
    train_label = [x for x in ActivityIdList if x["name"] == train_label]
    return train_label[0]['id']

In [196]:
# Create Graph per row of the House CSV
graphs = []
labels = []

# Combine Feature like this: Place_in_House,Type, Value for each node
for i in range(len(houseB)):
# for i in range(5000):
    
    feature  = []
    
    # Define Graph
    g = dgl.graph((u, v))

    # Add Features
    for j in range(len(nodes)):
        try:
            node_value = houseB.iloc[i, 4+j]        
        except:
            node_value = -1  
                            
        node_place_in_house = nodes.loc[j, 'place_in_house']
        node_type = nodes.loc[j, 'Type']
        feature.append([node_value, node_place_in_house, node_type])    
    g.ndata['feat'] = torch.tensor(feature)
    
    # Give Label
    labels.append(getIDFromClassName(houseB.iloc[i, 2]))
    graphs.append(g)

In [220]:
# Dataset Class
class GraphHouseDataset():    
    def __init__(self, graphs, labels):
        super(GraphHouseDataset, self).__init__()
        self.graphs = graphs
        self.labels = labels


    def __getitem__(self, idx):
        """ Get graph and label by index"""     
        return self.graphs[idx], self.labels[idx]

    def __len__(self):
        """Number of graphs in the dataset"""
        return len(self.graphs)

## "Start for some other dataset to check network"

In [229]:
import dgl.data
dataset = dgl.data.GINDataset('MUTAG', False)
def collate(samples):
    # The input `samples` is a list of pairs (graph, label).
    graphs, labels = map(list, zip(*samples))
    for g in graphs:
        # deal with node feats
        for key in g.node_attr_schemes().keys():
            g.ndata[key] = g.ndata[key].float()
        # no edge feats
    batched_graph = dgl.batch(graphs)
    labels = torch.tensor(labels)
    return batched_graph, labels

dataloader = DataLoader(dataset, batch_size=1, shuffle=False, collate_fn=collate)

## "End for some other dataset to check network"

In [230]:
# # Create DataLoaders
# dataset = GraphHouseDataset(graphs, labels)

# # create collate_fn
# def collate(samples):
#     # The input `samples` is a list of pairs (graph, label).
#     graphs, labels = map(list, zip(*samples))    
#     batched_graph = dgl.batch(graphs)
#     labels = torch.tensor(labels)
#     return batched_graph, labels


# dataloader = DataLoader(dataset, batch_size=1, shuffle=False, collate_fn=collate)


In [231]:
# [node_value, node_place_in_house, node_type]
torch.manual_seed(0)
import dgl.nn.pytorch as dglnn
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_classes):
        super(Classifier, self).__init__()
        self.conv1 = dglnn.GraphConv(in_dim, hidden_dim)
        self.conv2 = dglnn.GraphConv(hidden_dim, hidden_dim)
        self.classify = nn.Linear(hidden_dim, n_classes)

    def forward(self, g, h):
        # Apply graph convolution and activation.
        h = F.relu(self.conv1(g, h))
        h = F.relu(self.conv2(g, h))
        with g.local_scope():
            g.ndata['h'] = h
            # Calculate graph representation by average readout.
            hg = dgl.mean_nodes(g, 'h')
            return self.classify(hg)

In [232]:

import torch.nn.functional as F

In [237]:


# Featuresize, hiddenLayerSize, num_classes
model = Classifier(7, 10, 13)
opt = torch.optim.Adam(model.parameters())
for epoch in range(100):
    epoch_loss = 0
    acc= []
    for i, (batched_graphs, labels) in enumerate(dataloader):
        feats = batched_graphs.ndata['attr'].float()        
        logits = model(batched_graphs, feats)
        loss = F.cross_entropy(logits, labels)
        opt.zero_grad()
        pred = logits.argmax(1)  
#         if i % 500 == 0:
#             print(pred)
        train_acc = (pred == labels).sum().item()/len(pred)
        acc.append(train_acc)
        epoch_loss += loss
        loss.backward()
        opt.step()
    acc = np.array(acc).mean()    
    print('\n\nLoss %.4f, Train Acc %.4f \n\n' % (
        epoch_loss.item(),
        acc.item(),
    ))



Loss 445.6690, Train Acc 0.3936 




Loss 202.6177, Train Acc 0.6649 




Loss 136.8479, Train Acc 0.6649 




Loss 130.7178, Train Acc 0.6862 




Loss 131.1927, Train Acc 0.5851 




Loss 130.7272, Train Acc 0.5798 




Loss 130.1169, Train Acc 0.5798 




Loss 129.5733, Train Acc 0.5798 




Loss 129.1212, Train Acc 0.5798 




Loss 128.7089, Train Acc 0.5851 




Loss 128.3367, Train Acc 0.5904 




Loss 127.9451, Train Acc 0.6064 




Loss 127.5707, Train Acc 0.6277 




Loss 127.1402, Train Acc 0.6436 




Loss 126.7600, Train Acc 0.6436 




Loss 126.3372, Train Acc 0.6489 




Loss 126.0140, Train Acc 0.6596 




Loss 125.5622, Train Acc 0.6649 




Loss 125.1036, Train Acc 0.6649 




Loss 124.7540, Train Acc 0.6649 




Loss 124.3603, Train Acc 0.6649 




Loss 123.9012, Train Acc 0.6649 




Loss 123.4082, Train Acc 0.6649 




Loss 122.9898, Train Acc 0.6649 




Loss 122.5969, Train Acc 0.6649 




Loss 122.1880, Train Acc 0.6649 




Loss 121.6851, Train Acc 0.6649 





In [7]:
a = torch.tensor([[10,20,30,40], [10,20,30,40], [10,20,30,40]])
b = torch.tensor([10,20,31,41])

print(a.argmax(1))
# c = (a == b).sum().item()

# c1 = len(a)

# print(c/c1)


tensor([3, 3, 3])


In [19]:
a = [1,2,3.5]
np.array(a, dtype=float).mean()

2.1666666666666665

In [37]:
# import dgl.data
# dataset = dgl.data.GINDataset('MUTAG', False)


# dataset[0][0]

In [38]:
# def collate(samples):
#     graphs, labels = map(list, zip(*samples))
#     batched_graph = dgl.batch(graphs)
#     batched_labels = torch.tensor(labels)
#     return batched_graph, batched_labels

In [39]:
# from torch.utils.data import DataLoader
# dataloader = DataLoader(
#     dataset,
#     batch_size=1024,
#     collate_fn=collate,
#     drop_last=False,
#     shuffle=True)

In [74]:
class RGCN(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, rel_names):
        super().__init__()

        self.conv1 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(in_feats, hid_feats)
            for rel in rel_names}, aggregate='sum')
        self.conv2 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(hid_feats, out_feats)
            for rel in rel_names}, aggregate='sum')

    def forward(self, graph, inputs):
        # inputs is features of nodes
        h = self.conv1(graph, inputs)
        h = {k: F.relu(v) for k, v in h.items()}
        h = self.conv2(graph, h)
        return h

class HeteroClassifier(nn.Module):
    def __init__(self, G, in_dim, hidden_dim, n_classes, rel_names):
        super().__init__()                
        
        
        self.rgcn = RGCN(in_dim, hidden_dim, hidden_dim, rel_names)
        self.classify = nn.Linear(hidden_dim, n_classes)

    def forward(self, g):
        
        h = g.ndata
        h = self.rgcn(g, h)
        with g.local_scope():
            g.ndata = h
            # Calculate graph representation by average readout.
            hg = 0
            for ntype in g.ntypes:
                hg = hg + dgl.mean_nodes(g, 'h', ntype=ntype)
            return self.classify(hg)
        
        
model = HeteroClassifier(g, 5, 20, 5, 'connectedWith_1')
opt = torch.optim.Adam(model.parameters())
for epoch in range(20):
    for batched_graph, labels in dataloader:
        logits = model(batched_graph)
        loss = F.cross_entropy(logits, labels)
        opt.zero_grad()
        loss.backward()
        opt.step()

KeyError: 'connectedWith_1'

In [70]:
g.ndata

{'room': {'place_in_house': tensor([0, 1, 2, 3, 4])}, 'sensor': {'place_in_house': tensor([2, 0, 0, 2, 0, 0, 1, 1, 1, 0, 0, 1, 1, 2, 3, 2, 4, 3, 0, 0, 0, 0]), 'currentSensorvalues': tensor([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0])}}

In [53]:
import dgl.function as fn

class HeteroRGCNLayer(nn.Module):
    def __init__(self, in_size, out_size, etypes):
        super(HeteroRGCNLayer, self).__init__()
        # W_r for each relation
        self.weight = nn.ModuleDict({
                name : nn.Linear(in_size, out_size) for name in etypes
            })
        

    def forward(self, G, feat_dict):
        # The input is a dictionary of node features for each type
        funcs = {}
        for srctype, etype, dsttype in G.canonical_etypes:
            # Compute W_r * h            
            Wh = self.weight[etype](feat_dict[srctype])
            print(feat_dict[srctype])
            # Save it in graph for message passing
            G.nodes[srctype].data['Wh_%s' % etype] = Wh
            # Specify per-relation message passing functions: (message_func, reduce_func).
            # Note that the results are saved to the same destination feature 'h', which
            # hints the type wise reducer for aggregation.
            funcs[etype] = (fn.copy_u('Wh_%s' % etype, 'm'), fn.mean('m', 'h'))
        # Trigger message passing of multiple types.
        # The first argument is the message passing functions for each relation.
        # The second one is the type wise reducer, could be "sum", "max",
        # "min", "mean", "stack"
        G.multi_update_all(funcs, 'sum')
        # return the updated node feature dictionary
        return {ntype : G.nodes[ntype].data['h'] for ntype in G.ntypes}
    
    
class HeteroRGCN(nn.Module):
    def __init__(self, G, in_size, hidden_size, out_size):
        super(HeteroRGCN, self).__init__()
        # Use trainable node embeddings as featureless inputs.
        embed_dict = {ntype : nn.Parameter(torch.Tensor(G.number_of_nodes(ntype), in_size))
                      for ntype in G.ntypes}
        for key, embed in embed_dict.items():
            nn.init.xavier_uniform_(embed)
        self.embed = nn.ParameterDict(embed_dict)
        # create layers
        self.layer1 = HeteroRGCNLayer(in_size, hidden_size, G.etypes)
        self.layer2 = HeteroRGCNLayer(hidden_size, out_size, G.etypes)

    def forward(self, G):
        h_dict = self.layer1(G, self.embed)
        h_dict = {k : F.leaky_relu(h) for k, h in h_dict.items()}
        h_dict = self.layer2(G, h_dict)
        
        # get paper logits
        return h_dict
    

In [54]:
g = graphs[0]

In [55]:
model = HeteroRGCN(g, 5, 40, 20)

opt = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


In [56]:

best_val_acc = 0
best_test_acc = 0

for epoch in range(1):
    for g, labels in dataloader:    
        logits = model(g)
        loss = F.cross_entropy(logits, labels)
        opt.zero_grad()
        loss.backward()
        opt.step()        
        
        

Parameter containing:
tensor([[ 0.1982,  0.2756,  0.5585,  0.6418,  0.5529],
        [-0.0388,  0.5257,  0.7347,  0.3640, -0.3275],
        [ 0.4052, -0.3667, -0.4774,  0.3016, -0.2666],
        [-0.2678,  0.0197,  0.4296,  0.3594, -0.1597],
        [ 0.0518, -0.3857, -0.1902,  0.6816,  0.5124]], requires_grad=True)
Parameter containing:
tensor([[ 0.1982,  0.2756,  0.5585,  0.6418,  0.5529],
        [-0.0388,  0.5257,  0.7347,  0.3640, -0.3275],
        [ 0.4052, -0.3667, -0.4774,  0.3016, -0.2666],
        [-0.2678,  0.0197,  0.4296,  0.3594, -0.1597],
        [ 0.0518, -0.3857, -0.1902,  0.6816,  0.5124]], requires_grad=True)
Parameter containing:
tensor([[-0.2559,  0.1799,  0.1891,  0.2021, -0.1346],
        [ 0.1436,  0.0144,  0.4015, -0.2170, -0.4146],
        [-0.4713, -0.1568, -0.1289,  0.1052, -0.3894],
        [-0.3615,  0.4373,  0.2693, -0.0702, -0.1023],
        [ 0.2586, -0.2345, -0.0489,  0.2224, -0.1748],
        [-0.3641, -0.3023, -0.1429,  0.2922,  0.2682],
        [ 0.

In [38]:
g.etypes

['connectedWith', 'connectedWith', 'connectedWith']

# End

In [102]:
# with open('/home/harsh/Downloads/Thesis/Research/data/houseB/houseB.json') as f:
#     jsonFile = json.load(f)

In [103]:
# id =-1 
# for dictionary in jsonFile['sensorLocation']:
#     id+= 1
#     print(id,',', dictionary['name'],',', -1)

In [277]:
print(data.keys())

dict_keys(['__header__', '__version__', '__globals__', 'TvsP', 'PvsA', 'PvsV', 'AvsF', 'VvsC', 'PvsL', 'PvsC', 'A', 'C', 'F', 'L', 'P', 'T', 'V', 'PvsT', 'CNormPvsA', 'RNormPvsA', 'CNormPvsC', 'RNormPvsC', 'CNormPvsT', 'RNormPvsT', 'CNormPvsV', 'RNormPvsV', 'CNormVvsC', 'RNormVvsC', 'CNormAvsF', 'RNormAvsF', 'CNormPvsL', 'RNormPvsL', 'stopwords', 'nPvsT', 'nT', 'CNormnPvsT', 'RNormnPvsT', 'nnPvsT', 'nnT', 'CNormnnPvsT', 'RNormnnPvsT', 'PvsP', 'CNormPvsP', 'RNormPvsP'])


In [273]:
print(type(data['PvsA']))
print('#Papers:', data['PvsA'].shape[0])
print('#Authors:', data['PvsA'].shape[1])
print('#Links:', data['PvsA'].nnz)

<class 'scipy.sparse.csc.csc_matrix'>
#Papers: 12499
#Authors: 17431
#Links: 37055


In [283]:
x = data['PvsA'][0, :]
print(x)

  (0, 4610)	1.0
  (0, 5298)	1.0
  (0, 12982)	1.0


In [47]:
pa_g = dgl.heterograph({('paper', 'written-by', 'author') : data['PvsA'].nonzero()})

In [48]:
print('Node types:', pa_g.ntypes)
print('Edge types:', pa_g.etypes)
print('Canonical edge types:', pa_g.canonical_etypes)

Node types: ['author', 'paper']
Edge types: ['written-by']
Canonical edge types: [('paper', 'written-by', 'author')]


In [49]:
print(pa_g.number_of_nodes('paper'))
# Canonical edge type name can be shortened to only one edge type name if it is
# uniquely distinguishable.
print(pa_g.number_of_edges(('paper', 'written-by', 'author')))
print(pa_g.number_of_edges('written-by'))
print(pa_g.successors(1, etype='written-by'))  # get the authors that write paper #1
print(pa_g.number_of_edges())  # Only one edge type, the edge type argument could be omitted


12499
37055
37055
tensor([3532, 6421, 8516, 8560])
37055


In [8]:
pp_g = dgl.heterograph({('paper', 'citing', 'paper') : data['PvsP'].nonzero()})
# equivalent (shorter) API for creating homogeneous graph
pp_g = dgl.from_scipy(data['PvsP'])

# All the ntype and etype arguments could be omitted because the behavior is unambiguous.
print(pp_g.number_of_nodes())
print(pp_g.number_of_edges())
print(pp_g.successors(3))

12499
30789
tensor([1361, 2624, 8670, 9845])


In [279]:
data['PvsA']

<1x17431 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Column format>

In [288]:
G = dgl.heterograph({
        ('paper', 'written-by', 'author') : data['PvsA'].nonzero(),
        ('author', 'writing', 'paper') : data['PvsA'].transpose().nonzero(),
        ('paper', 'citing', 'paper') : data['PvsP'].nonzero(),
        ('paper', 'cited', 'paper') : data['PvsP'].transpose().nonzero(),
        ('paper', 'is-about', 'subject') : data['PvsL'].nonzero(),
        ('subject', 'has', 'paper') : data['PvsL'].transpose().nonzero(),
    })




In [66]:
# Draw the metagraph using graphviz.
import pygraphviz as pgv
def plot_graph(nxg):
    ag = pgv.AGraph(strict=False, directed=True)
    for u, v, k in nxg.edges(keys=True):
        ag.add_edge(u, v, label=k)
    ag.layout('dot')
    ag.draw('graph.png')

plot_graph(G.metagraph())

In [68]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

pvc = data['PvsC'].tocsr()

In [72]:
# find all papers published in KDD, ICML, VLDB
c_selected = [0, 11, 13]  # KDD, ICML, VLDB
p_selected = pvc[:, c_selected].tocoo()
# generate labels
labels = pvc.indices

In [ ]:
labels[labels == 11] = 1
labels[labels == 13] = 2
labels = torch.tensor(labels).long()

# generate train/val/test split
pid = p_selected.row
shuffle = np.random.permutation(pid)
train_idx = torch.tensor(shuffle[0:800]).long()
val_idx = torch.tensor(shuffle[800:900]).long()
test_idx = torch.tensor(shuffle[900:]).long()

In [18]:
import dgl.function as fn

class HeteroRGCNLayer(nn.Module):
    def __init__(self, in_size, out_size, etypes):
        super(HeteroRGCNLayer, self).__init__()
        # W_r for each relation
        self.weight = nn.ModuleDict({
                name : nn.Linear(in_size, out_size) for name in etypes
            })

    def forward(self, G, feat_dict):
        # The input is a dictionary of node features for each type
        funcs = {}
        for srctype, etype, dsttype in G.canonical_etypes:
            # Compute W_r * h
            print(self.weight[etype])
            Wh = self.weight[etype](feat_dict[srctype])
            # Save it in graph for message passing
            G.nodes[srctype].data['Wh_%s' % etype] = Wh
            # Specify per-relation message passing functions: (message_func, reduce_func).
            # Note that the results are saved to the same destination feature 'h', which
            # hints the type wise reducer for aggregation.
            funcs[etype] = (fn.copy_u('Wh_%s' % etype, 'm'), fn.mean('m', 'h'))
        # Trigger message passing of multiple types.
        # The first argument is the message passing functions for each relation.
        # The second one is the type wise reducer, could be "sum", "max",
        # "min", "mean", "stack"
        G.multi_update_all(funcs, 'sum')
        # return the updated node feature dictionary
        return {ntype : G.nodes[ntype].data['h'] for ntype in G.ntypes}
    
    
    
class HeteroRGCN(nn.Module):
    def __init__(self, G, in_size, hidden_size, out_size):
        super(HeteroRGCN, self).__init__()
        # Use trainable node embeddings as featureless inputs.
        embed_dict = {ntype : nn.Parameter(torch.Tensor(G.number_of_nodes(ntype), in_size))
                      for ntype in G.ntypes}
        for key, embed in embed_dict.items():
            nn.init.xavier_uniform_(embed)
        self.embed = nn.ParameterDict(embed_dict)
        # create layers
        self.layer1 = HeteroRGCNLayer(in_size, hidden_size, G.etypes)
        self.layer2 = HeteroRGCNLayer(hidden_size, out_size, G.etypes)

    def forward(self, G):
        h_dict = self.layer1(G, self.embed)
        h_dict = {k : F.leaky_relu(h) for k, h in h_dict.items()}
        h_dict = self.layer2(G, h_dict)
        # get paper logits
        return h_dict['paper']
    
    
    
model = HeteroRGCN(G, 10, 40, 20)

opt = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


NameError: name 'G' is not defined

In [321]:

best_val_acc = 0
best_test_acc = 0

for epoch in range(100):
    logits = model(G)
    # The loss is computed only for labeled nodes.
    loss = F.cross_entropy(logits[train_idx], labels[train_idx])

    pred = logits.argmax(1)
    train_acc = (pred[train_idx] == labels[train_idx]).float().mean()
    val_acc = (pred[val_idx] == labels[val_idx]).float().mean()
    test_acc = (pred[test_idx] == labels[test_idx]).float().mean()

    if best_val_acc < val_acc:
        best_val_acc = val_acc
        best_test_acc = test_acc

    opt.zero_grad()
    loss.backward()
    opt.step()

    if epoch % 5 == 0:
        print('Loss %.4f, Train Acc %.4f, Val Acc %.4f (Best %.4f), Test Acc %.4f (Best %.4f)' % (
            loss.item(),
            train_acc.item(),
            val_acc.item(),
            best_val_acc.item(),
            test_acc.item(),
            best_test_acc.item(),
        ))

KeyError: 'writing'